# Debt Market Model

$$
\Delta{t} = t_{k+1} - t_{k}\\
{Q}_{k+1} = {Q}_k + v_1 - v_2 - v_3\\
{D_1}_{k+1} = {D_1}_k + u_1 - u_2 - u_3\\
w_3 = u_3 \cdot \frac{w_2}{u_2}\\
w_1 = [(1+\beta_k)^{\Delta{t}}-1]({D_1}_k+{D_2}_k)\\
{D_2}_{k+1} = {D_2}_k + w_1 - w_2 - w_3\\
{R}_{k+1} = {R}_k + w_2\\
$$

## First phase
* Debt market state -> ETH price changes (exogenous) -> exogenous u,v -> endogenous w -> mutates system state

## Second phase
* APT model, arbitragers act -> u,v activity (to remove diversifiable risk) -> results in change to both debt market and secondary market -> stability controller updates redemption rate and price

In [ ]:
from shared import *

In [ ]:
# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

In [ ]:
import radCAD as rc
from radCAD import Model, Simulation

In [ ]:
import scipy.stats as sts
import numpy as np
import datetime as dt

In [ ]:
def resolve_time_passed(params, substep, state_history, state):
    seconds = 3600
    
    return {'seconds_passed': seconds}

def store_timedelta(params, substep, state_history, state, policy_input):

    value = policy_input['seconds_passed']
    key = 'timedelta'

    return key,value

def update_timestamp(params, substep, state_history, state, policy_input):

    seconds = policy_input['seconds_passed']
    value = state['timestamp'] + dt.timedelta(seconds = int(seconds))
    key = 'timestamp'

    return key,value

def update_cumulative_time(params, substep, state_history, state, policy_input):
    seconds = policy_input['seconds_passed']
    
    return 'cumulative_time', state['cumulative_time'] + seconds

In [ ]:
def process_raw_results(raw_results):
    df = pd.DataFrame(raw_results)
    max_substep = max(df.substep)
    is_droppable = (df.substep != max_substep)
    is_droppable &= (df.substep != 0)
    df = df.loc[~is_droppable]
    return df

In [ ]:
def p_resolve_eth_price(params, substep, state_history, state):
    base_var = params['eth_market_std']
    variance = float(base_var * state['timedelta'] / 3600.0)
    delta_eth_price = sts.norm.rvs(loc=0, scale=variance)
    
    return {'delta_eth_price': delta_eth_price}

def s_update_eth_price(params, substep, state_history, state, policy_input):
    eth_price = state['eth_price']
    delta_eth_price = policy_input['delta_eth_price']
    
    return 'eth_price', eth_price + delta_eth_price

def s_update_redemption_price(params, substep, state_history, state, policy_input):
    eth_collateral = state['eth_collateral']
    eth_price = state['eth_price']
    collateral_value = eth_collateral * eth_price
    
    principal_debt = state['principal_debt']
    redemption_price = collateral_value / principal_debt
    
    return 'redemption_price', redemption_price

In [ ]:
def p_open_cdps(params, substep, state_history, state):
    base_var = 2.0
    variance = float(base_var * state['timedelta'] / 3600.0)
    random_state = params['random_state']
    rvs = sts.norm.rvs(loc=0, scale=variance, random_state=random_state)
    v1 = max(rvs, 0) # total Eth value of new CDP
    
    # cdps = state['cdps']

    collateralization_ratio = params['collateralization_ratio']
    collateral_value = v1 * state['eth_price']
    redemption_price = state['redemption_price']
    u1 = (collateral_value / redemption_price) / collateralization_ratio
    
    cumulative_time = state['cumulative_time']
    # Hourly activity
    if cumulative_time % 3600 == 0:
        return {'delta_v1': v1, 'delta_u1': u1}
    else:
        return {'delta_v1': 0, 'delta_u1': 0}

def s_update_eth_collateral(params, substep, state_history, state, policy_input):
    eth_locked = state['eth_locked']
    eth_freed = state['eth_freed']
    eth_bitten = state['eth_bitten']
    
    return 'eth_collateral', eth_locked - eth_freed - eth_bitten

def s_update_principal_debt(params, substep, state_history, state, policy_input):
    rai_drawn = state['rai_drawn']
    rai_wiped = state['rai_wiped']
    rai_bitten = state['rai_bitten']
    
    return 'principal_debt', rai_drawn - rai_wiped - rai_bitten

def s_update_eth_locked(params, substep, state_history, state, policy_input):
    eth_locked = state['eth_locked']
    delta_v1 = policy_input['delta_v1']
    
    return 'eth_locked', eth_locked + delta_v1

def s_update_rai_drawn(params, substep, state_history, state, policy_input):
    rai_drawn = state['rai_drawn']
    delta_u1 = policy_input['delta_u1']
    
    return 'rai_drawn', rai_drawn + delta_u1
    
def s_update_accrued_interest(params, substep, state_history, state, policy_input):
    previous_accrued_interest = state['accrued_interest']
    principal_debt = state['principal_debt']
    
    stability_fee = state['stability_fee']
    timedelta = state['timedelta']
    
    accrued_interest = ((1 + stability_fee)**timedelta - 1) * (principal_debt + previous_accrued_interest)
    return 'accrued_interest', previous_accrued_interest + accrued_interest

In [ ]:
eth_collateral = 100.0
eth_price = 386.71

collateralization_ratio = 1.5 # 150%
collateral_value = eth_collateral * eth_price
redemption_price = 2.0
principal_debt = (collateral_value / redemption_price) / collateralization_ratio

print(f'''
{principal_debt}
{eth_collateral}
''')

In [ ]:
SIMULATION_TIMESTEPS = 365 * 24
MONTE_CARLO_RUNS = 1

In [ ]:
initial_state = {
    'timedelta': 0, # seconds
    'cumulative_time': 0, # seconds
    'timestamp': dt.datetime.strptime('12/18/18', '%m/%d/%y'), #datetime
    #'cdps': cdps,
    'eth_price': eth_price, # dollars
    # v
    'eth_collateral': eth_collateral, # Q
    'eth_locked': eth_collateral, # v1
    'eth_freed': 0, # v2
    'eth_bitten': 0, # v3 "liquidated"
    # u
    'principal_debt': principal_debt, # D1
    'rai_drawn': principal_debt, # u1 "minted"
    'rai_wiped': 0, # u2 "burned" in repayment
    'rai_bitten': 0, # "burned" in liquidation
    # w
    'accrued_interest': 0, # D2
    'system_revenue': 0, # R
    # Average CDP duration == 3 months: https://www.placeholder.vc/blog/2019/3/1/maker-network-report
    'average_debt_age': 3 * (30 * 24 * 3600), # delta t (seconds)
    'stability_fee': 0.15 / (30 * 24 * 3600), # per second interest rate (15% per month)
    'interest_wiped': 0, # w2, interest repaid - in practice acrues to MKR holders, because interest is actually acrued by burning MKR
    'interest_bitten': 0, # w3
    'redemption_price': redemption_price, # dollars
}

parameters = {
    'eth_market_std': [1],
    'collateralization_ratio': [collateralization_ratio], # %
    'random_state': [np.random.RandomState(seed=i) for i in range(MONTE_CARLO_RUNS)]
}

partial_state_update_blocks = [
    {
        'details': '''
            This block observes (or samples from data) the amount of time passed between events
        ''',
        'policies': {
            'time_process': resolve_time_passed
        },
        'variables': {
            'timedelta': store_timedelta,
            'timestamp': update_timestamp,
            'cumulative_time': update_cumulative_time
        }
    },
    {
        'details': '''
            Update debt market state
        ''',
        'policies': {},
        'variables': {
            'eth_collateral': s_update_eth_collateral,
            'principal_debt': s_update_principal_debt,
            'accrued_interest': s_update_accrued_interest
        }
    },
    {
        'policies': {},
        'variables': {
            'redemption_price': s_update_redemption_price
        }
    },
    {
        'details': '''
            Exogenous ETH price process
        ''',
        'policies': {
            'exogenous_eth_process': p_resolve_eth_price,
        },
        'variables': {
            'eth_price': s_update_eth_price
        }
    },
    {
        'details': '''
            Exogenous u,v activity
        ''',
        'policies': {
            'open_cdps': p_open_cdps
        },
        'variables': {
            'eth_locked': s_update_eth_locked,
            'rai_drawn': s_update_rai_drawn
        }
    },
    {
        'details': '''
            Endogenous w activity
        ''',
        'policies': {},
        'variables': {
        }
    }
]

In [ ]:
# model = Model(initial_state=initial_state, psubs=partial_state_update_blocks, params=parameters)
# simulation = Simulation(model=model, timesteps=SIMULATION_TIMESTEPS, runs=MONTE_CARLO_RUNS)

# import time
# start = time.time()
# raw_result = rc.run([simulation])
# end = time.time()
# print(end - start)

# data = process_raw_results(raw_result)

In [ ]:
from cadCAD import configs
del configs[:]

experiment = Experiment()

sim_config = config_sim({
    'N': MONTE_CARLO_RUNS,
    'T': range(SIMULATION_TIMESTEPS),
    'M': parameters
})

experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

exec_context = ExecutionContext()

simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()

In [ ]:
simulation_result = pd.DataFrame(raw_result)

In [ ]:
simulation_result.plot(x='timestamp', y=['eth_price'])

In [ ]:
simulation_result.plot(x='timestamp', y=['redemption_price'])

In [ ]:
simulation_result.plot(x='timestamp', y=['eth_collateral'])

In [ ]:
simulation_result.plot(x='timestamp', y=['eth_locked', 'eth_freed', 'eth_bitten'])

In [ ]:
simulation_result.plot(x='timestamp', y=['principal_debt'])

In [ ]:
simulation_result.plot(x='timestamp', y=['rai_drawn', 'rai_wiped', 'rai_bitten'])

In [ ]:
simulation_result.plot(x='timestamp', y=['accrued_interest'])

In [ ]:
simulation_result['collateralization_ratio'] = (simulation_result.eth_collateral * simulation_result.eth_price) / (simulation_result.principal_debt * simulation_result.redemption_price)
simulation_result.plot(x='timestamp', y=['collateralization_ratio'])

<center><img src="diagrams/debt_dynamics.png" alt="Debt dynamics" width="60%"/>

In [ ]:
import xarray as xr

xcdp = xr.Dataset(
        data_vars={'locked': (['id', 'timestep'], [(0.0, 0.0, 0.0, 0), (0.0, 0.0, 0.0, 0)]),
#                    'drawn': (['id'], [0, 1]),
#                    'wiped': (['id'], [0, 1])
                  },
        coords={'id': [10, 20],
                'timestep': [0, 1]
               })

# da = xr.DataArray([[0.0, 0.0, 0.0, 0]], dims=('id', 'timestep', 'locked', 'drawn', 'wiped', 'age'))

# da

xcdp['locked']

In [ ]:
import numpy as np

# CDP: (locked, drawn, wiped, age)

total_cdps = 10_000
cdps = np.empty(total_cdps, dtype=object)
cdps[:] = [(0.0, 0.0, 0.0, 0) for i in range(total_cdps)]

print(cdps.shape)

cdps[0] = (eth_collateral, principal_debt, 0.0, 0)
cdps

In [ ]:
x = np.array(None); x[()] = (0.0, 0.0, 0.0, 0)

cdps[np.where(cdps == x)[0]] = (1, 2, 3, 4)
cdps